# 1 预处理数据

## 1.1 将串口值整理成合适的格式

In [2]:
import numpy as np

# 定义读取文件并处理数据的函数
def read_and_process_data(file_path):
    data = []

    # 打开并读取文件
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 临时存储每组16个adc值
    temp_group = []

    for line in lines:
        # 分割行内容
        parts = line.split('=')
        if len(parts) == 2:
            # 提取并转换为浮点数
            value = float(parts[1].strip())
            temp_group.append(value)

            # 每组16个adc值完成后，添加到data中
            if len(temp_group) == 16:
                data.append(temp_group)
                temp_group = []

    # 转换为numpy数组
    data_array = np.array(data)
    return data_array

# 文件路径
'''
没有遮挡 (None)
石头 (Rock)
剪刀 (Scissors)
布 (Paper)
'''

None_file_path = './None.txt'
Rock_file_path = './Rock.txt'
Scissors_file_path = './Scissors.txt'
Paper_file_path = './Paper.txt'

# 读取并处理数据
None_data = read_and_process_data(None_file_path)
Rock_data = read_and_process_data(Rock_file_path)
Scissors_data = read_and_process_data(Scissors_file_path)
Paper_data = read_and_process_data(Paper_file_path)

# 输出数据以供检查
# 只输出前5组数据
print("None_data (前5组):\n", None_data[:5])
print("\nRock_data (前5组):\n", Rock_data[:5])
print("\nScissors_data (前5组):\n", Scissors_data[:5])
print("\nPaper_data (前5组):\n", Paper_data[:5])



None_data (前5组):
 [[3855. 3830. 3792. 3823. 3803. 3829. 3809. 3772. 3823. 3782. 3770. 3793.
  3769. 3786. 3743. 3744.]
 [3872. 3812. 3801. 3826. 3812. 3805. 3810. 3771. 3848. 3767. 3763. 3786.
  3773. 3792. 3733. 3730.]
 [3862. 3803. 3793. 3830. 3796. 3816. 3810. 3769. 3848. 3763. 3763. 3789.
  3774. 3763. 3733. 3732.]
 [3865. 3815. 3795. 3832. 3787. 3822. 3809. 3768. 3838. 3775. 3760. 3765.
  3763. 3771. 3733. 3718.]
 [3860. 3806. 3801. 3824. 3791. 3817. 3812. 3769. 3840. 3767. 3763. 3785.
  3773. 3792. 3723. 3728.]]

Rock_data (前5组):
 [[3839. 3805. 3780. 3807. 3780. 3789. 3808. 3755. 3823.  827.  875. 3556.
  3669. 3615.  784. 3575.]
 [3834. 3757. 3732. 3743. 3716.  909. 2822. 3679. 3815.  849.  864. 3572.
  3669. 3663.  783. 3578.]
 [3842. 3760. 3727. 3760. 3725.  973. 2567. 3672. 3815.  829.  849. 3568.
  3669. 3675.  790. 3584.]
 [3840. 3747. 3754. 3764. 3703. 1019. 2641. 3690. 3813.  747.  810. 3572.
  3677. 3661.  800. 3573.]
 [3823. 3735. 3752. 3760. 3718. 1048. 2580. 3678. 381

## 数据增强：拓展到四个方向

In [3]:
# 初始方向
'''
0 1 2 3
8 9 10 11
4 5 6 7
12 13 14 15
'''
import numpy as np

# 定义一个函数进行旋转操作
def rotate_matrix(matrix, angle):
    if angle == 90:
        return np.rot90(matrix, k=3)
    elif angle == 180:
        return np.rot90(matrix, k=2)
    elif angle == 270:
        return np.rot90(matrix, k=1)
    else:
        return matrix

# 将一维数组转换为4x4矩阵
def to_4x4_matrix(array):
    matrix = np.zeros((4, 4))
    matrix[0, :] = array[0:4]
    matrix[1, :] = array[8:12]
    matrix[2, :] = array[4:8]
    matrix[3, :] = array[12:16]
    return matrix

# 将4x4矩阵转换回一维数组
def from_4x4_matrix(matrix):
    array = np.zeros(16)
    array[0:4] = matrix[0, :]
    array[8:12] = matrix[1, :]
    array[4:8] = matrix[2, :]
    array[12:16] = matrix[3, :]
    return array

# 将数据转换为4x4矩阵，并生成旋转后的数据
def rotate_data(original_data):
    expanded_data = []
    for group in original_data:
        matrix = to_4x4_matrix(group)
        expanded_data.append(from_4x4_matrix(matrix))  # 原始方向
        expanded_data.append(from_4x4_matrix(rotate_matrix(matrix, 90)))  # 旋转90度
        expanded_data.append(from_4x4_matrix(rotate_matrix(matrix, 180)))  # 旋转180度
        expanded_data.append(from_4x4_matrix(rotate_matrix(matrix, 270)))  # 旋转270度
    return np.array(expanded_data)

# 生成拓展方向并合并后的数据
expanded_None_data = rotate_data(None_data)
expanded_Rock_data = rotate_data(Rock_data)
expanded_Scissors_data = rotate_data(Scissors_data)
expanded_Paper_data = rotate_data(Paper_data)



## Keams算法求聚类中心

In [4]:
'''
Rock_data 标签是 0
Scissors_data 标签是 1
Paper_data 标签是 2
'''
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

# 标签定义（排除 None_data）
labels = np.array([0] * len(expanded_Rock_data) + 
                  [1] * len(expanded_Scissors_data) + 
                  [2] * len(expanded_Paper_data))

# 合并所有数据（排除 None_data）
data = np.vstack([expanded_Rock_data, expanded_Scissors_data, expanded_Paper_data])

def compute_accuracy(predicted_labels, true_labels):
    """计算准确率"""
    return accuracy_score(true_labels, predicted_labels)

def classify(input_data, cluster_centers, output_distances=False):
    """根据聚类中心对输入数据进行分类"""
    input_data = np.array(input_data)  # 确保输入数据是numpy数组
    input_data = input_data.reshape(1, -1)  # 将输入数据转换为二维数组
    distances = np.linalg.norm(input_data - cluster_centers, axis=1)
    if output_distances:
        print(f"距离: {distances}")
    return np.argmin(distances), distances

def label_to_word(label):
    """映射类别编号到实际单词"""
    word_map = {0: 'Rock', 1: 'Scissors', 2: 'Paper'}
    return word_map.get(label, 'Unknown')

# 设定准确率阈值
accuracy_threshold = 0.95
max_iterations = 1000 # 最大迭代次数

for iteration in range(max_iterations):
    # K-means 聚类
    n_clusters = 3
    kmeans = KMeans(n_clusters=n_clusters, random_state=iteration)
    kmeans.fit(data)
    
    # 获取聚类中心
    cluster_centers = kmeans.cluster_centers_
    
    # 预测训练数据的标签
    predicted_labels = np.array([classify(d, cluster_centers)[0] for d in data])
    
    # 计算准确率
    accuracy = compute_accuracy(predicted_labels, labels)
   
    if accuracy >= accuracy_threshold:
        break

# 输出最终的聚类中心
print(f"最终聚类中心: {cluster_centers}")

# 测试一个输入数据
Input_file_path = './Input.txt'
input_data = read_and_process_data(Input_file_path)
predicted_label, distances = classify(input_data, cluster_centers, output_distances=True)

# 输出类别序号和对应的单词
predicted_word = label_to_word(predicted_label)
print(f"输入数据的分类标签: {predicted_label}")
print(f"输入数据的分类单词: {predicted_word}")




最终聚类中心: [[3685.81461676 2361.55793226 2398.58823529 3623.04099822 3585.92691622
  1968.82174688 1608.16934046 3610.14438503 3579.26737968 1573.21033868
  1980.67557932 3628.56684492 3636.65418895 2410.41889483 2380.05525847
  3706.92869875]
 [3641.67158672 3643.9298893  3646.26937269 3758.28782288 2293.78597786
  1348.15498155 2806.79335793 1447.67896679 1292.50922509 2615.43173432
  1456.73431734 2430.17712177 3736.11808118 3688.66420664 3687.68265683
  3698.05535055]
 [3631.26689189 3556.02702703 3583.10472973 3643.47297297 2471.62837838
  1768.22297297 1116.31418919 3299.77702703 3500.36148649 1357.77027027
  1646.34797297 2265.94256757 3637.96959459 3521.86824324 3480.91216216
  3539.62837838]]
距离: [3073.6637628  6807.01220659 5153.20932146]
输入数据的分类标签: 0
输入数据的分类单词: Rock
